In [ ]:
import pandas as pd

In [ ]:
pip install pandas_ta

In [ ]:
pip install yfinance

In [ ]:
pip install backtesting

In [ ]:
import yfinance as yf

ho scaricato i dati da https://www.myfxbook.com/forex-market/currencies/AUDUSD-historical-data. ti mando per mail il file che ho utilizzato per questo notebook. ma è poca roba. è da rifare con yfinance.

In [ ]:
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
from os.path import exists
import pandas as pd
#from backtesting import Strategy
#from backtesting import Backtest
import pandas_ta as ta

# Old

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Financial works/AUDUSD_historical_data.csv"

In [ ]:
df = pd.read_csv(DATA_PATH, sep=';')

poteva essere una funzione con un for ma non avevo voglia

In [ ]:
def div(value):
    if value >= 1:
        value /= 1000
    return value

# To perform operation row wise in an efficient way.
df.High = df.loc[:,'High'].apply(lambda row: div(row))

In [ ]:
def div(value):
    if value >= 1:
        value /= 1000
    return value

# To perform operation row wise in an efficient way.
df.Low = df.loc[:,'Low'].apply(lambda row: div(row))

In [ ]:
def div(value):
    if value >= 1:
        value /= 1000
    return value

# To perform operation row wise in an efficient way.
df.Open = df.loc[:,'Open'].apply(lambda row: div(row))

In [ ]:
def div(value):
    if value >= 1:
        value /= 1000
    return value

# To perform operation row wise in an efficient way.
df.Close = df.loc[:,'Close'].apply(lambda row: div(row))

# New

In [ ]:
forex_data = yf.download('AUDUSD=X', start='2021-01-02', end='2022-11-08', interval='1h')

[*********************100%***********************]  1 of 1 completed


In [ ]:
forex_data.index = pd.to_datetime(forex_data.index)

In [ ]:
forex_data

,Open,High,Low,Close,Adj Close,Volume
2021-01-04 00:00:00,0.771230,0.771784,0.768362,0.768817,0.768817,0
2021-01-04 01:00:00,0.769089,0.772022,0.768965,0.771772,0.771772,0
2021-01-04 02:00:00,0.771837,0.772499,0.771182,0.771539,0.771539,0
2021-01-04 03:00:00,0.771635,0.772141,0.771391,0.771754,0.771754,0
2021-01-04 04:00:00,0.771760,0.772022,0.770066,0.770333,0.770333,0
...,...,...,...,...,...,...
2022-11-07 19:00:00,0.648126,0.648382,0.647698,0.647798,0.647798,0
2022-11-07 20:00:00,0.647899,0.648508,0.647618,0.648008,0.648008,0
2022-11-07 21:00:00,0.648008,0.648298,0.647819,0.648139,0.648139,0
2022-11-07 22:00:00,0.647962,0.648311,0.647626,0.647811,0.647811,0


In [ ]:
df = forex_data

In [ ]:
df = df.drop(columns = ["Volume"])

In [ ]:
#This function uses the pandas ta library to calculate the EMAs for the OHLC data.

def calculate_emas(df, window):
    df['Open_EMA'] = ta.ema(df['Open'], length=window)
    df['Close_EMA'] = ta.ema(df['Close'], length=window)
    df['High_EMA'] = ta.ema(df['High'], length=window)
    df['Low_EMA'] = ta.ema(df['Low'], length=window)
    return df

#These functions are used to detect the bullish and bearish trend reversals.

def bullish_change(df):
    df['bullish_change'] = 0
    for i in range(1, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]        
        idx = df.index[i]
        bullish_change = prev_1['Close'] <= prev_1['Open'] and \
                         current['Close'] > current['Open']
        if bullish_change:
            df.loc[idx, 'bullish_change'] = 1
    return df

def bearish_change(df):
    df['bearish_change'] = 0
    for i in range(1, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        idx = df.index[i]
        bearish_change = prev_1['Close'] >= prev_1['Open'] and \
                         current['Close'] < current['Open']
        if bearish_change:
            df.loc[idx, 'bearish_change'] = 1
    return df

#And there the smoothed Haikin-Ashi function, which uses the EMAs instead of the regular OHLC prices.

def transform_to_heikin_ashi_smoothed(df):
    '''
    Function to transform dataframe to Heikin Ashi DataFrame
    The input dataframe columns should be named 'Open', 'High', 'Low', 'Close'.
    '''
    df_temp = df[['Open', 'High', 'Low', 'Close', 'Open_EMA', 'High_EMA','Low_EMA','Close_EMA']]    
    ha_df = df_temp.copy()    # Transform Close column
    ha_df['Close'] = round(((df_temp['Open_EMA'] + df_temp['High_EMA'] + df_temp['Low_EMA'] + df_temp['Close_EMA']) / 4), 2)    # Transform Open column
    for i in range(len(df_temp)):
        if i == 0:
            ha_df.iat[0, 0] = round(((df_temp['Open_EMA'].iloc[0] + df_temp['Close_EMA'].iloc[0]) / 2), 2)
        else:
            ha_df.iat[i, 0] = round(((ha_df.iat[i - 1, 0] + ha_df.iat[i - 1, 3]) / 2), 2)
    # Transform High
    ha_df['High'] = ha_df.loc[:, ['Open_EMA', 'Close_EMA']].join(ha_df['High_EMA']).max(axis=1)    
    #  Transform Low column
    ha_df['Low'] = ha_df.loc[:, ['Open_EMA', 'Close_EMA']].join(ha_df['Low_EMA']).min(axis=1)
    return ha_df

#And finally, the main function, which loads a OHLC price data file, calculates the EMAs, transforms the dataframe into the smoothed Heikin-Ashi values and then calculates the trend reversal indicators.

# MAIN
#if name == '__main__':
    #  Symbols: DNUT, ETSY, FTCH, PYPL
   #  Calculate EMAs for smoothing
df2 = calculate_emas(df, 1)    #  Drop nans

df2.dropna(inplace=True)    #  Transform to Heikin Ashi


HAdf = transform_to_heikin_ashi_smoothed(df2)

print(HAdf)

In [ ]:
# HAdf.insert(0, value=df['Date'], column='Date')

In [ ]:
# HAdf = HAdf.set_index('Date')

In [ ]:
# HAdf = HAdf.sort_index()

In [ ]:
HAdf

,Open,High,Low,Close,Open_EMA,High_EMA,Low_EMA,Close_EMA
2021-01-04 00:00:00,0.77,0.771784,0.768362,0.77,0.771230,0.771784,0.768362,0.768817
2021-01-04 01:00:00,0.77,0.772022,0.768965,0.77,0.769089,0.772022,0.768965,0.771772
2021-01-04 02:00:00,0.77,0.772499,0.771182,0.77,0.771837,0.772499,0.771182,0.771539
2021-01-04 03:00:00,0.77,0.772141,0.771391,0.77,0.771635,0.772141,0.771391,0.771754
2021-01-04 04:00:00,0.77,0.772022,0.770066,0.77,0.771760,0.772022,0.770066,0.770333
...,...,...,...,...,...,...,...,...
2022-11-07 19:00:00,0.64,0.648382,0.647698,0.65,0.648126,0.648382,0.647698,0.647798
2022-11-07 20:00:00,0.64,0.648508,0.647618,0.65,0.647899,0.648508,0.647618,0.648008
2022-11-07 21:00:00,0.64,0.648298,0.647819,0.65,0.648008,0.648298,0.647819,0.648139
2022-11-07 22:00:00,0.64,0.648311,0.647626,0.65,0.647962,0.648311,0.647626,0.647811


In [ ]:
df = df.set_index('Date')
df = df.sort_index()

In [ ]:
df

,Open,High,Low,Close,Adj Close,Open_EMA,Close_EMA,High_EMA,Low_EMA
2021-01-04 00:00:00,0.771230,0.771784,0.768362,0.768817,0.768817,0.771230,0.768817,0.771784,0.768362
2021-01-04 01:00:00,0.769089,0.772022,0.768965,0.771772,0.771772,0.769089,0.771772,0.772022,0.768965
2021-01-04 02:00:00,0.771837,0.772499,0.771182,0.771539,0.771539,0.771837,0.771539,0.772499,0.771182
2021-01-04 03:00:00,0.771635,0.772141,0.771391,0.771754,0.771754,0.771635,0.771754,0.772141,0.771391
2021-01-04 04:00:00,0.771760,0.772022,0.770066,0.770333,0.770333,0.771760,0.770333,0.772022,0.770066
...,...,...,...,...,...,...,...,...,...
2022-11-07 19:00:00,0.648126,0.648382,0.647698,0.647798,0.647798,0.648126,0.647798,0.648382,0.647698
2022-11-07 20:00:00,0.647899,0.648508,0.647618,0.648008,0.648008,0.647899,0.648008,0.648508,0.647618
2022-11-07 21:00:00,0.648008,0.648298,0.647819,0.648139,0.648139,0.648008,0.648139,0.648298,0.647819
2022-11-07 22:00:00,0.647962,0.648311,0.647626,0.647811,0.647811,0.647962,0.647811,0.648311,0.647626


In [ ]:
import plotly.graph_objects as go

fig1 = go.Figure(data=[go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)])

fig1.update_layout(yaxis_range = [0.6,0.85], 
           title = 'Candlestick Chart: AUDUSD', 
           xaxis_title = 'Date', 
           yaxis_title = 'Price')
fig1.show()

In [ ]:
fig2 = go.Figure(data=[go.Candlestick(x=HAdf.index,
                open=HAdf.Open_EMA,
                high=HAdf.High_EMA,
                low=HAdf.Low_EMA,
                close=HAdf.Close_EMA)] )


fig2.update_layout(yaxis_range = [0.60,0.85], 
          title = 'Heikin Ashi Chart: AUSUSD', 
          xaxis_title = 'Date', 
          yaxis_title = 'Price')
fig2.show()

In [ ]:
df

,Open,High,Low,Close,Adj Close,Open_EMA,Close_EMA,High_EMA,Low_EMA,bullish_change,bearish_change
2021-01-04 00:00:00,0.771230,0.771784,0.768362,0.768817,0.768817,0.771230,0.768817,0.771784,0.768362,False,False
2021-01-04 01:00:00,0.769089,0.772022,0.768965,0.771772,0.771772,0.769089,0.771772,0.772022,0.768965,False,False
2021-01-04 02:00:00,0.771837,0.772499,0.771182,0.771539,0.771539,0.771837,0.771539,0.772499,0.771182,False,False
2021-01-04 03:00:00,0.771635,0.772141,0.771391,0.771754,0.771754,0.771635,0.771754,0.772141,0.771391,True,False
2021-01-04 04:00:00,0.771760,0.772022,0.770066,0.770333,0.770333,0.771760,0.770333,0.772022,0.770066,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07 19:00:00,0.648126,0.648382,0.647698,0.647798,0.647798,0.648126,0.647798,0.648382,0.647698,False,False
2022-11-07 20:00:00,0.647899,0.648508,0.647618,0.648008,0.648008,0.647899,0.648008,0.648508,0.647618,True,False
2022-11-07 21:00:00,0.648008,0.648298,0.647819,0.648139,0.648139,0.648008,0.648139,0.648298,0.647819,False,False
2022-11-07 22:00:00,0.647962,0.648311,0.647626,0.647811,0.647811,0.647962,0.647811,0.648311,0.647626,False,True


# Lorenzo strategy

In [ ]:
def bullish_change(df):
    df['bullish_change'] = False
    for i in range(2, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        idx = df.index[i]
        bullish_change = prev_1['Close_EMA'] < current['Open_EMA'] and \
                         current['Close_EMA'] > current['Open_EMA']
        if bullish_change:
            df.loc[idx, 'bullish_change'] = True
    return df
def bearish_change(df):
    df['bearish_change'] = False
    for i in range(2, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        idx = df.index[i]
        bearish_change = prev_1['Close_EMA'] > current['Open_EMA'] and \
                         current['Close_EMA'] < current['Open_EMA']
        if bearish_change:
            df.loc[idx, 'bearish_change'] = True
    return df


In [ ]:
df = bullish_change(df)
df = bearish_change(df)

In [ ]:
def lorenzo_strategy_buy(df):
    df['buy'] = False
    for i in range(3, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        prev_2 = df.iloc[i - 2, :]
        idx = df.index[i]
        buy = prev_2['bullish_change'] == True and \
                         prev_1['Close'] > prev_2['Close']
        if buy:
            df.loc[idx, 'buy'] = True
    return df

def lorenzo_strategy_sell(df):
    df['sell'] = False
    for i in range(3, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        prev_2 = df.iloc[i - 2, :]
        idx = df.index[i]
        sell = prev_2['bearish_change'] == True and \
                         prev_1['Close'] < prev_2['Close']
        if sell:
            df.loc[idx, 'sell'] = True
    return df

In [ ]:
df = lorenzo_strategy_buy(df)
df = lorenzo_strategy_sell(df)

In [ ]:
from numpy import nan
def stop_loss_long(df):
    df['stop_loss_long'] = nan
    for i in range(3, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        prev_2 = df.iloc[i - 2, :]
        idx = df.index[i]
        stop_loss_long = current['buy'] == True

        if stop_loss_long:
            df.loc[idx, 'stop_loss_long'] = prev_2['Low']
    return df

def stop_loss_short(df):
    df['stop_loss_short'] = nan
    for i in range(3, df.shape[0]):
        current = df.iloc[i, :]
        prev_1 = df.iloc[i - 1, :]
        prev_2 = df.iloc[i - 2, :]
        idx = df.index[i]
        stop_loss_short = current['sell'] == True

        if stop_loss_short:
            df.loc[idx, 'stop_loss_short'] = prev_2['High']
    return df

In [ ]:
df = stop_loss_long(df)
df = stop_loss_short(df)

In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'bullish_change', 'bearish_change',
       'buy', 'sell', 'stop_loss_long', 'stop_loss_short', 'stop_loss'],
      dtype='object')

In [ ]:
df = df.drop(columns=['Adj Close', 'Open_EMA', 'Close_EMA', 'High_EMA', 'Low_EMA'])

In [ ]:
df = df.drop(columns=['stop_loss_long', 'stop_loss_short'])

In [ ]:
df['stop_loss'] = df['stop_loss_long'].fillna(df['stop_loss_short'])

In [ ]:
df.to_csv('AUDUSD_per_backtest.csv')

# Backtesting

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

## Strategia per Backtesting  

definire buy su buy = true e sell;  
short selling  
inserire stop loss

In [ ]:
class MixedStrategy(Strategy):
    long_hold = 0
    i = 0
    def init(self):
        pass

    def next(self):
        
        if self['buy'] == True:
          self.position.close()

          self.buy()
        

In [ ]:
def run_backtest(df):
    # If exclusive orders (each new order auto-closes previous orders/position),
    # cancel all non-contingent orders and close all open trades beforehand
    bt = Backtest(df, MixedStrategy, cash=10000, commission=3.45, trade_on_close=True, exclusive_orders=True, hedging=False)
    stats = bt.run()
    print(stats)
    bt.plot()

In [ ]:
# MAIN
if __name__ == '__main__':
    #  Add bullish/bearish change indicators
    df = bullish_change(df)
    df = bearish_change(df)
    #  Run backtest
    run_backtest(df)

Start                     2021-01-04 00:00:00
End                       2022-11-07 23:00:00
Duration                    672 days 23:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              -15.763238
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`



In [ ]:
df

,Open,High,Low,Close,bullish_change,bearish_change,buy,sell,stop_loss
2021-01-04 00:00:00,0.771230,0.771784,0.768362,0.768817,False,False,False,False,NaN
2021-01-04 01:00:00,0.769089,0.772022,0.768965,0.771772,False,False,False,False,NaN
2021-01-04 02:00:00,0.771837,0.772499,0.771182,0.771539,False,True,False,False,NaN
2021-01-04 03:00:00,0.771635,0.772141,0.771391,0.771754,True,False,False,False,NaN
2021-01-04 04:00:00,0.771760,0.772022,0.770066,0.770333,False,True,False,False,NaN
...,...,...,...,...,...,...,...,...,...
2022-11-07 19:00:00,0.648126,0.648382,0.647698,0.647798,False,True,True,False,0.646178
2022-11-07 20:00:00,0.647899,0.648508,0.647618,0.648008,True,False,False,False,NaN
2022-11-07 21:00:00,0.648008,0.648298,0.647819,0.648139,False,False,False,False,NaN
2022-11-07 22:00:00,0.647962,0.648311,0.647626,0.647811,False,True,True,False,0.647618
